In [48]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_S_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from Algorithmes.music import estimate_angles
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from torchsummary import summary
from numpy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import math
import random
from sklearn.metrics import mean_squared_error
from Algorithmes.music import generate_steering_vector
from Plots.draw_plot import plot_single_music

In [116]:
nbSources = 1 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 5 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels



#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = nbSensors
T = 500
correlation_List = [] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.
SNR_TRAIN = 15
Q = 8

In [117]:
import numpy as np

def calculate_music_spectrum(R_hat, nbSensors, nbSources, theta_range):
    eigenvalues, eigenvectors = np.linalg.eigh(R_hat)
    noise_subspace = eigenvectors[:, :nbSensors - nbSources]
    music_spectrum = np.zeros(len(theta_range))

    for i, theta in enumerate(theta_range):
        steering_vector = generate_steering_vector(nbSensors, theta)
        music_spectrum[i] = 1 / np.abs(np.conj(steering_vector).T @ noise_subspace @ noise_subspace.conj().T @ steering_vector)

    return music_spectrum



In [118]:
def generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q):
    training_data_sets = [[] for _ in range(Q)]
    theta_start, theta_final = -45, 45  # Full DOA angle range
    N = 2**12  # Total number of points in the full range
    L = N // Q  # Number of points in each subregion

    for j_alpha in range(J_alpha):
        thetaList = np.random.uniform(theta_start, theta_final, nbSources).tolist()

        for j_beta in range(J_beta):
            varList = np.ones(nbSources).tolist()
            correlation_List = np.zeros(nbSources * (nbSources - 1) // 2).tolist()
            X = generate_X_matrix(nbSources, nbSensors, T, thetaList, varList, correlation_List, SNR_TRAIN)
            
            R_hat = generate_R_hat(X)
            full_theta = np.linspace(theta_start, theta_final, N)
            full_music_spectrum = calculate_music_spectrum(R_hat, nbSensors, nbSources, full_theta)

            for q in range(Q):
                # Sampling MUSIC spectrum for the q-th subregion
                start_index = q * L
                end_index = start_index + L
                pq = full_music_spectrum[start_index:end_index]
                
                input_data = generate_R_hat_with_phase(X)
                output_data = pq
                training_data_sets[q].append((input_data, output_data))                

    return training_data_sets



# Exemple d'utilisation de la fonction
J_alpha = 100  # Nombre d'ensembles d'angles DOA à générer
J_beta = 100  # Nombre d'itérations pour chaque ensemble d'angles DOA
training_dataset = generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q)


[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]

In [97]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class DeepMusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, output_data = self.data[idx]
        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(output_data, dtype=torch.float32)

def create_loaders_for_subregion(data, batch_size=64, validation_split=0.1, test_split=0.1):
    # Séparation des données en ensembles d'entraînement, de validation et de test
    train_data, test_data = train_test_split(data, test_size=test_split, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=validation_split, random_state=42)

    # Création des instances de DeepMusicDataset
    train_dataset = DeepMusicDataset(train_data)
    val_dataset = DeepMusicDataset(val_data)
    test_dataset = DeepMusicDataset(test_data)

    # Création des DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Création des DataLoader pour chaque sous-région
loaders_per_subregion = [create_loaders_for_subregion(training_dataset[q]) for q in range(Q)]

In [119]:

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=4096)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Softmax layer 
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = self.fc1(x)
        
        # Apply the dropout layer
        x = self.dropout(x)

        #Apply the softmax layer
        x = self.softmax(x)
        
        return x

model = DeepMusicModel()
model

DeepMusicModel(
  (conv1): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=20736, out_features=4096, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)

In [109]:
#Définition du device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [110]:
#Définition des hyperparamètres, choisis en fonction de l'article DOA estimator

learning_rate = 0.001
num_epochs = 10 # à monter à 200 pour le training final selon l'article
loss_fn = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.MSELoss()

In [111]:
def train_loop(model, criterion, optimizer, train_loader, device):
    model.train()  # Mettre le modèle en mode entraînement
    total_loss = 0
    

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss


In [112]:
import torch
import numpy as np
from scipy.signal import find_peaks
import torch.nn.functional as F

def val_loop(model, val_loader, peak_tolerance=5,loss_fn=nn.CrossEntropyLoss()):
    model.eval()  # Set the model to evaluation mode
    peak_similarity_score = 0
    total_mse = 0
    total_samples = 0
    test_loss = 0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, targets in val_loader:
            # Forward pass
            outputs = model(inputs)
            test_loss += loss_fn(outputs, targets).item()
            # Calculate MSE for each batch
            mse = F.mse_loss(outputs, targets)
            total_mse += mse.item()

            # Find peaks in the predictions and targets
            for output, target in zip(outputs, targets):
                predicted_peaks, _ = find_peaks(output.cpu().numpy())
                target_peaks, _ = find_peaks(target.cpu().numpy())

                # Compare the sets of peaks
                similarity = sum(1 for pred_peak in predicted_peaks if np.any(np.abs(target_peaks - pred_peak) <= peak_tolerance))
                peak_similarity_score += similarity / len(predicted_peaks) if predicted_peaks.size > 0 else 0
                total_samples += 1

    avg_peak_similarity = peak_similarity_score / total_samples if total_samples > 0 else 0
    avg_mse = total_mse / total_samples if total_samples > 0 else 0
    print(f'Average Peak Similarity Score: {avg_peak_similarity:.4f}')
    print(f'Average MSE: {avg_mse:.4f}')
    print(f'Avg loss: {test_loss: >8f}')


In [129]:
import torch
import numpy as np
from scipy.signal import find_peaks
import torch.nn.functional as F

def train_and_evaluate_spectrum(loaders_per_subregion, epochs, peak_tolerance=5):
    models = []
    train_loss_list = []
    val_loss_list = []
    val_accuracy_list = []

    # Training configuration
    initial_lr = 0.01
    lr_decay_factor = 0.5
    lr_decay_epoch = 10
    early_stopping_patience = 3

    for train_loader, val_loader, _ in loaders_per_subregion:
        model = DeepMusicModel()
        optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=0.9)
        criterion = torch.nn.MSELoss()

        best_val_accuracy = 0
        epochs_no_improve = 0

        for epoch in range(epochs):
            # Adjust learning rate
            if epoch % lr_decay_epoch == 0 and epoch > 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= lr_decay_factor

            # Training loop
            model.train()
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

            # Validation loop
            model.eval()
            val_loss, val_accuracy = 0, 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
                    # Implement your method to calculate accuracy
                    val_accuracy += calculate_accuracy(outputs, targets)

            avg_val_loss = val_loss / len(val_loader)
            avg_val_accuracy = val_accuracy / len(val_loader)
            val_loss_list.append(avg_val_loss)
            val_accuracy_list.append(avg_val_accuracy)

            # Early stopping
            if avg_val_accuracy > best_val_accuracy:
                best_val_accuracy = avg_val_accuracy
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == early_stopping_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        models.append(model)

    # Combine predictions for the complete spectrum and evaluate
    complete_predicted_spectrum, complete_true_spectrum = [], []
    for model, (_, _, test_loader) in zip(models, loaders_per_subregion):
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                complete_predicted_spectrum.extend(outputs.cpu().numpy().flatten())
                complete_true_spectrum.extend(targets.cpu().numpy().flatten())

    mse = F.mse_loss(torch.tensor(complete_predicted_spectrum), torch.tensor(complete_true_spectrum))
    predicted_peaks, _ = find_peaks(complete_predicted_spectrum)
    true_peaks, _ = find_peaks(complete_true_spectrum)
    peak_similarity_score = sum(1 for pred_peak in predicted_peaks if np.any(np.abs(true_peaks - pred_peak) <= peak_tolerance)) / len(predicted_peaks) if predicted_peaks.size > 0 else 0

    print(f'Average Training Loss: {np.mean(train_loss_list):.4f}')
    print(f'Average Validation Loss: {np.mean(val_loss_list):.4f}')
    print(f'Complete Spectrum MSE: {mse:.4f}')
    print(f'Peak Similarity Score: {peak_similarity_score:.4f}')

# Example usage
train_and_construct_spectrum(loaders_per_subregion, epochs=100, peak_tolerance=10)


Complete Spectrum MSE: 193.4067
Peak Similarity Score: 0.0335
